# Building up

In [1]:
import numpy as np
import time
time_start = time.time()
X = np.genfromtxt('00_Data/train_X_dog_cat.csv', delimiter=',')
y = np.genfromtxt('00_Data/train_y_dog_cat.csv', delimiter=',')
print(X.shape)

n = X.shape[0]
d = X.shape[1]
print(y.shape)
time_end = time.time()
print("Done loading training data, using time:(min)", (time_end-time_start)/60)

(1953, 3072)
(1953,)
Done loading training data, using time:(min) 0.06361965735753378


In [2]:
X_test = np.genfromtxt('00_Data/test_X_dog_cat.csv', delimiter=',')
y_test = np.genfromtxt('00_Data/test_y_dog_cat.csv', delimiter=',')
m = X_test.shape[0]
print(m)

2000


In [3]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler().fit(X)
X = scaler.transform(X)
X_test = scaler.transform(X_test)


In [9]:
def linear_kernel(x1,x2):
    # x1 = (n,d) X2 = (m,d)
    return(np.dot(x1,x2.T))

def inhomogeneous_poly_kernel(x1,x2):
    n = x1.shape[0]
    m = x2.shape[0]
    return np.power(np.ones(shape=(n,m)) + np.dot(x1,x2.T), 5)

                  
def gaussian_kernel(x1,x2,sigma=100):
    n = x1.shape[0]
    m = x2.shape[0]
    K = np.zeros(shape=(n,m))
    for i in range(n):
        for j in range(m):
            K[i][j] = gaussian(x1[i],x2[j],sigma)
    return (K)

def gaussian(x,y,sigma):
    return np.exp((-(np.linalg.norm(x-y,2)**2))/(2*sigma**2))

In [5]:
def evaluation(ans, y):
    right = 0
    num = ans.shape[0]
    for i in range(ans.shape[0]):
        if (ans[i] - 0.5)*y[i] > 0:
            right += 1
    return(right/float(num))

In [6]:
def tolerable(delta,tol):
    from numpy import linalg as LA
    if LA.norm(delta,1) > tol:
        return False
    else:
        return True

In [14]:
def probability(alpha, Ki):
    s = np.dot(alpha, Ki)
    return(1/(1+np.exp(-s)))

def train(K,y,max_iter,batch_size,regu_para, eta,tol):
    time_start = time.time()
    #alpha = np.random.uniform(size=(n))
    alpha = np.zeros(n)
    for t in range(max_iter):
        samples = np.random.permutation(n)[:batch_size]
        g = regu_para * alpha
        #g = np.zeros(n)
        for i in samples:
            pi = probability(alpha, K[i])
            g[i] = g[i] + (pi - (y[i]+1)/2.0)
        alpha = alpha - eta*g
        if tolerable(g, tol):
            break
        if t % 2000 == 0:
            print("============== %d =================" % t)
            print("alpha:", alpha)
            print("train_accuracy:",evaluation(probability(alpha,K), y))
    time_end = time.time()
    print("Done training, using time:(min)", 
          (time_end-time_start)/60)
    return(alpha)

In [15]:
def experiment(kernel, X,y,X_test,y_test,regu_para, eta,tol= 0.1, max_iter=50000, batch_size=640):
    K = kernel(X,X)
    K_test = kernel(X,X_test)
    print("Done kernel generating")
    alpha = train(K,y, max_iter,batch_size,regu_para, eta,tol)
    print("Done training")
    print(alpha)
    print("training result")
    ans = probability(alpha,K)
    print(ans)
    print(evaluation(ans, y))
    print(y)
    print("test result")
    ans = probability(alpha,K_test)
    print(ans)
    print(evaluation(ans, y_test))
    print(y_test)

# Linear kernel exps

In [16]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0, eta=0.01, tol=0.1)

Done kernel generating
============== 0 =================
alpha: [ 0.     0.    -0.005 ...  0.     0.     0.   ]
train_accuracy: 0.5314900153609831


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


============== 2000 =================
alpha: [-2.95068478 -3.42       -0.54270089 ...  3.28086929  3.26
  3.24      ]
train_accuracy: 0.5468509984639017
============== 4000 =================
alpha: [-5.96121963 -6.61       -0.54270089 ...  6.37086929  6.92
  6.55      ]
train_accuracy: 0.5683563748079877
============== 6000 =================
alpha: [-7.50183508 -9.81       -0.54270089 ...  9.64086929 10.14
  9.56      ]
train_accuracy: 0.5857654889912954
============== 8000 =================
alpha: [ -8.74803732 -13.04        -0.54270206 ...  12.96086929  13.3
  12.83      ]
train_accuracy: 0.5949820788530465
============== 10000 =================
alpha: [-10.18366344 -16.18        -0.55269902 ...  16.20086929  16.51
  16.28      ]
train_accuracy: 0.5985663082437276
============== 12000 =================
alpha: [-11.92667826 -19.57        -0.55269902 ...  19.35987454  19.83
  19.49000455]
train_accuracy: 0.6185355862775218
============== 14000 =================
alpha: [-13.92835032 -22

In [ ]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0, eta=0.001, tol=0.1)

In [ ]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0, eta=0.0001, tol=0.1)

In [ ]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=0, eta=0.01, tol=0.01)

In [68]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=10, eta=0.01, tol=0.1)

Done kernel generating
============== 0 =================
alpha: [ 0.    -0.005 -0.005 ...  0.     0.005  0.005]
train_accuracy: 0.5202252944188428


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


============== 500 =================
alpha: [-0.01736876 -0.02710565 -0.02290689 ...  0.02543125  0.02729629
  0.0130005 ]
train_accuracy: 0.5202252944188428
============== 1000 =================
alpha: [-0.02963044 -0.0341762  -0.01427181 ...  0.03124184  0.02068479
  0.01910747]
train_accuracy: 0.5202252944188428
============== 1500 =================
alpha: [-0.02045605 -0.01691791 -0.03268337 ...  0.03607512  0.02870882
  0.03065018]
train_accuracy: 0.5202252944188428
============== 2000 =================
alpha: [-0.03952263 -0.02507891 -0.03158553 ...  0.02732721  0.03463066
  0.02127832]
train_accuracy: 0.5202252944188428
============== 2500 =================
alpha: [-0.02963365 -0.02325948 -0.02993138 ...  0.01601174  0.02494453
  0.03606875]
train_accuracy: 0.5202252944188428
============== 3000 =================
alpha: [-0.02209631 -0.02119482 -0.03434269 ...  0.02445605  0.02154613
  0.02743588]
train_accuracy: 0.5202252944188428
============== 3500 =================
alpha: [-

============== 26500 =================
alpha: [-0.01930345 -0.02324162 -0.0094324  ...  0.02471421  0.04439771
  0.0206016 ]
train_accuracy: 0.5202252944188428
============== 27000 =================
alpha: [-0.01564781 -0.02533036 -0.03757566 ...  0.01782466  0.02072666
  0.0278958 ]
train_accuracy: 0.5202252944188428
============== 27500 =================
alpha: [-0.02599332 -0.03017529 -0.02924788 ...  0.02251209  0.02627011
  0.02871434]
train_accuracy: 0.5202252944188428
============== 28000 =================
alpha: [-0.01891928 -0.02926585 -0.03118648 ...  0.02396996  0.01760371
  0.01731038]
train_accuracy: 0.5202252944188428
============== 28500 =================
alpha: [-0.01938013 -0.02739281 -0.02304781 ...  0.02303721  0.03702199
  0.02623346]
train_accuracy: 0.5202252944188428
============== 29000 =================
alpha: [-0.02770608 -0.0379979  -0.04188599 ...  0.02214793  0.01897246
  0.03558965]
train_accuracy: 0.5202252944188428
============== 29500 =================
a

In [76]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=30, eta=0.01, tol=0.1)

Done kernel generating
============== 0 =================
alpha: [ 0.    -0.005 -0.005 ...  0.005  0.     0.   ]
train_accuracy: 0.5202252944188428


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


============== 500 =================
alpha: [-0.00188142 -0.00288345 -0.00713507 ...  0.00986007  0.00411522
  0.0100699 ]
train_accuracy: 0.5202252944188428
============== 1000 =================
alpha: [-0.01292425 -0.01258557 -0.00185759 ...  0.00453667  0.01342899
  0.01339869]
train_accuracy: 0.5202252944188428
============== 1500 =================
alpha: [-0.01662485 -0.01779168 -0.01464934 ...  0.00484549  0.0094036
  0.01015824]
train_accuracy: 0.5202252944188428
============== 2000 =================
alpha: [-0.00040712 -0.01216403 -0.01330473 ...  0.0014913   0.01160008
  0.00844602]
train_accuracy: 0.5202252944188428
============== 2500 =================
alpha: [-0.00820659 -0.01086404 -0.00249737 ...  0.0004141   0.00730654
  0.00858693]
train_accuracy: 0.5202252944188428
============== 3000 =================
alpha: [-0.01308258 -0.008907   -0.00314135 ...  0.00763708  0.00398494
  0.00761218]
train_accuracy: 0.5202252944188428
============== 3500 =================
alpha: [-0

============== 26000 =================
alpha: [-0.01495376 -0.00370702 -0.00382401 ...  0.00996903  0.00188832
  0.00073668]
train_accuracy: 0.5202252944188428
============== 26500 =================
alpha: [-0.0107598  -0.00864492 -0.00855154 ...  0.01125503  0.0010922
  0.00514952]
train_accuracy: 0.5202252944188428
============== 27000 =================
alpha: [-0.00412903 -0.01454699 -0.01586669 ...  0.0011608   0.00872242
  0.00561497]
train_accuracy: 0.5202252944188428
============== 27500 =================
alpha: [-0.01046147 -0.01331605 -0.00731133 ...  0.00426138  0.01237308
  0.01018945]
train_accuracy: 0.5202252944188428
============== 28000 =================
alpha: [-0.00906469 -0.01507201 -0.00764023 ...  0.01300309  0.00564725
  0.00859824]
train_accuracy: 0.5202252944188428
============== 28500 =================
alpha: [-0.01882195 -0.0053448  -0.00687144 ...  0.0040738   0.00669703
  0.00491304]
train_accuracy: 0.5202252944188428
============== 29000 =================
al

In [67]:
experiment(linear_kernel, X, y, X_test, y_test, regu_para=1, eta=0.01, tol=0.1)

Done kernel generating
============== 0 =================
alpha: [-0.005 -0.005 -0.005 ...  0.     0.005  0.   ]
train_accuracy: 0.5202252944188428


/home/y247xie/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


============== 500 =================
alpha: [-0.00070162 -0.27387453 -0.24696995 ...  0.25188899  0.2597418
  0.2500594 ]
train_accuracy: 0.5192012288786483
============== 1000 =================
alpha: [-4.60997561e-06 -2.82247503e-01 -2.54775806e-01 ...  2.95018054e-01
  2.96378451e-01  2.25204803e-01]
train_accuracy: 0.5197132616487455
============== 1500 =================
alpha: [-3.44633033e-08 -2.46297601e-01 -2.54201986e-01 ...  2.53170196e-01
  2.45024818e-01  2.66168575e-01]
train_accuracy: 0.5217613927291347
============== 2000 =================
alpha: [-0.00033473 -0.22549923 -0.22446756 ...  0.27803965  0.22546866
  0.21020405]
train_accuracy: 0.5207373271889401
============== 2500 =================
alpha: [-2.20133156e-06 -2.50232655e-01 -2.75417379e-01 ...  2.93534146e-01
  1.91656413e-01  2.42693027e-01]
train_accuracy: 0.5212493599590374
============== 3000 =================
alpha: [-0.00776671 -0.23365963 -0.25755962 ...  0.23832151  0.25902235
  0.2506014 ]
train_accur

============== 24500 =================
alpha: [-0.00931506 -0.19934185 -0.30542715 ...  0.25099686  0.23656815
  0.24178419]
train_accuracy: 0.5217613927291347
============== 25000 =================
alpha: [-6.14883014e-05 -2.51216377e-01 -2.66034160e-01 ...  2.60704197e-01
  2.50718135e-01  2.61636894e-01]
train_accuracy: 0.5192012288786483
============== 25500 =================
alpha: [-4.04007842e-07 -2.34214377e-01 -3.09052916e-01 ...  2.45074788e-01
  2.48328841e-01  2.35771861e-01]
train_accuracy: 0.5217613927291347
============== 26000 =================
alpha: [-2.67927436e-09 -2.80396784e-01 -2.50442091e-01 ...  2.79060241e-01
  2.32750498e-01  2.31733409e-01]
train_accuracy: 0.5212493599590374
============== 26500 =================
alpha: [-0.00265366 -0.29053678 -0.23468226 ...  0.2609087   0.27730826
  0.27997515]
train_accuracy: 0.5202252944188428
============== 27000 =================
alpha: [-1.74358446e-05 -2.54137361e-01 -2.91652744e-01 ...  2.78808888e-01
  2.71167447e

============== 48000 =================
alpha: [-0.01037697 -0.24560188 -0.27623091 ...  0.30050621  0.24845756
  0.23798603]
train_accuracy: 0.5227854582693292
============== 48500 =================
alpha: [-6.81816907e-05 -2.94995708e-01 -2.67778739e-01 ...  2.61552657e-01
  2.60872793e-01  2.43792254e-01]
train_accuracy: 0.5202252944188428
============== 49000 =================
alpha: [-7.74308900e-07 -2.71459268e-01 -2.82736412e-01 ...  2.24725417e-01
  2.18926153e-01  2.84839714e-01]
train_accuracy: 0.5207373271889401
============== 49500 =================
alpha: [-0.00101118 -0.24282342 -0.30925578 ...  0.2498697   0.25135568
  0.22315969]
train_accuracy: 0.5202252944188428
Done training, using time:(min) 4.450113276640574
Done training
[-0.00449588 -0.24318714 -0.28520803 ...  0.26406931  0.25330019
  0.24116423]
training result
[1.27030439e-62 1.00000000e+00 1.00000000e+00 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]
0.4910394265232975
[-1. -1. -1. ...  1.  1.  1.]
test re

# gaussian kernel exps

In [ ]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0, eta=0.01, tol=0.1)

In [ ]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=1, eta=0.001, tol=0.1)

In [ ]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=10, eta=0.001, tol=0.1)

In [ ]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0, eta=0.001, tol=0.1)

Done kernel generating
============== 0 =================
alpha: [-0.0005 -0.0005 -0.0005 ...  0.      0.      0.    ]
train_accuracy: 0.5202252944188428
============== 2000 =================
alpha: [-0.30253807 -0.32660729 -0.28760531 ...  0.33562117  0.31724887
  0.31546451]
train_accuracy: 0.590373783922171
============== 4000 =================
alpha: [-0.55316154 -0.66522004 -0.56998026 ...  0.69632066  0.63049064
  0.64175101]
train_accuracy: 0.5898617511520737
============== 6000 =================
alpha: [-0.7810752  -1.0050726  -0.84945169 ...  1.08057279  0.9328701
  0.95046721]
train_accuracy: 0.6026625704045059
============== 8000 =================
alpha: [-1.00154074 -1.34999888 -1.10021251 ...  1.443936    1.23202291
  1.24027757]
train_accuracy: 0.6052227342549923
============== 10000 =================
alpha: [-1.22547769 -1.70319388 -1.33040963 ...  1.80466983  1.52373209
  1.56337989]
train_accuracy: 0.5965181771633384
============== 12000 =================
alpha: [-1.44

In [ ]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0, eta=0.01, tol=0.01)

In [88]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0.01, eta=0.01, tol=0.01)

Done kernel generating
============== 0 =================
alpha: [ 0.    -0.005 -0.005 ...  0.005  0.     0.   ]
train_accuracy: 0.5647721454173067
============== 500 =================
alpha: [-0.77712428 -1.14391195 -0.94787028 ...  1.1973717   1.06150385
  1.10153621]
train_accuracy: 0.8766001024065541
============== 1000 =================
alpha: [-1.27974536 -2.11568956 -1.67766695 ...  2.13560717  1.82121079
  2.12191288]
train_accuracy: 0.9395801331285202
============== 1500 =================
alpha: [-1.64023431 -2.91948587 -2.26689255 ...  2.90888084  2.37559792
  3.01050622]
train_accuracy: 0.9687660010240655
============== 2000 =================
alpha: [-1.91788784 -3.57094776 -2.71633844 ...  3.55832335  2.85705189
  3.75956743]
train_accuracy: 0.978494623655914
============== 2500 =================
alpha: [-2.1581808  -4.17908705 -3.10488558 ...  4.03098553  3.29297508
  4.38345834]
train_accuracy: 0.985663082437276
============== 3000 =================
alpha: [-2.33737305 -4

============== 27500 =================
alpha: [-3.20697097 -8.24965277 -5.18116665 ...  6.22288317  5.89471506
  8.51713636]
train_accuracy: 1.0
============== 28000 =================
alpha: [-3.21096235 -8.26492581 -5.1863231  ...  6.21539253  5.89724115
  8.51300597]
train_accuracy: 1.0
============== 28500 =================
alpha: [-3.21004525 -8.24725091 -5.18733648 ...  6.21433828  5.89438267
  8.50738302]
train_accuracy: 1.0
============== 29000 =================
alpha: [-3.20521001 -8.25839077 -5.19019844 ...  6.2173424   5.86769153
  8.51932405]
train_accuracy: 1.0
============== 29500 =================
alpha: [-3.2149526  -8.26290211 -5.18792828 ...  6.19548213  5.85706531
  8.54973556]
train_accuracy: 1.0
============== 30000 =================
alpha: [-3.20745301 -8.26432468 -5.20341548 ...  6.19910548  5.88350588
  8.57223455]
train_accuracy: 1.0
============== 30500 =================
alpha: [-3.20936548 -8.28402346 -5.1962919  ...  6.21374464  5.88598842
  8.57803011]
train

In [ ]:
experiment(gaussian_kernel, X, y, X_test, y_test, regu_para=0, eta=0.0001, tol=0.01)

Done kernel generating
============== 0 =================
alpha: [-5.e-05 -5.e-05  0.e+00 ...  0.e+00  5.e-05  0.e+00]
train_accuracy: 0.5514592933947773
============== 500 =================
alpha: [-0.01226114 -0.01221707 -0.01264711 ...  0.01297682  0.01314281
  0.01224404]
train_accuracy: 0.6134152585765489
============== 1000 =================
alpha: [-0.02357217 -0.02458649 -0.0249565  ...  0.02693484  0.02641222
  0.02499511]
train_accuracy: 0.6139272913466461
============== 1500 =================
alpha: [-0.03379141 -0.03676273 -0.03804824 ...  0.04142833  0.03921462
  0.03710884]
train_accuracy: 0.6277521761392729
============== 2000 =================
alpha: [-0.04460262 -0.05032332 -0.04952317 ...  0.05466267  0.05238204
  0.04786029]
train_accuracy: 0.6400409626216078
============== 2500 =================
alpha: [-0.05471722 -0.0631394  -0.06083681 ...  0.06782688  0.06571849
  0.05934486]
train_accuracy: 0.6436251920122887
============== 3000 =================
alpha: [-0.064

============== 26000 =================
alpha: [-0.47072194 -0.62888474 -0.55237963 ...  0.69786559  0.62590524
  0.5831985 ]
train_accuracy: 0.8100358422939068
============== 26500 =================
alpha: [-0.47848168 -0.64077071 -0.56221207 ...  0.71035706  0.63593806
  0.59458845]
train_accuracy: 0.8120839733742959
============== 27000 =================
alpha: [-0.48633928 -0.65260178 -0.57186781 ...  0.72260575  0.64625376
  0.60663537]
train_accuracy: 0.8136200716845878


# inhomogeneous kernel exps

In [ ]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=0, eta=0.01, tol=0.1)

In [ ]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=1, eta=0.01, tol=0.1)

In [ ]:
experiment(inhomogeneous_poly_kernel, X, y, X_test, y_test, regu_para=10, eta=0.01, tol=0.1)